# Feature Transformation with Amazon a SageMaker Processing Job and Apache Spark

Typically a machine learning (ML) process consists of few steps. First, gathering data with various ETL jobs, then pre-processing the data, featurizing the dataset by incorporating standard techniques or prior knowledge, and finally training an ML model using an algorithm.

Often, distributed data processing frameworks such as Apache Spark are used to pre-process data sets in order to prepare them for training. In this notebook we'll use Amazon SageMaker Processing, and leverage the power of Apache Spark in a managed SageMaker environment to run our processing workload.

![](img/prepare_dataset_bert.png)

![](img/processing.jpg)


# Setup Environment

Let's start by specifying:
* The S3 bucket and prefixes that you use for training and model data. Use the default bucket specified by the Amazon SageMaker session.
* The IAM role ARN used to give processing and training access to the dataset.

In [1]:
import sagemaker
from time import gmtime, strftime
import boto3

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
region = boto3.Session().region_name

# Setup Input Data

In [2]:
# Inputs
s3_input_data = 's3://{}/amazon-reviews-pds/tsv/'.format(bucket)
print(s3_input_data)

s3://sagemaker-us-west-2-250107111215/amazon-reviews-pds/tsv/


In [3]:
!aws s3 ls $s3_input_data

2020-08-22 17:41:31   18997559 amazon_reviews_us_Digital_Software_v1_00.tsv.gz
2020-08-22 17:41:44   27442648 amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz


# Build a Spark Docker Image to Run the Processing Job

An example Spark container is included in the `./container` directory of this example. The container handles the bootstrapping of all Spark configuration, and serves as a wrapper around the `spark-submit` CLI. At a high level the container provides:
* A set of default Spark/YARN/Hadoop configurations
* A bootstrapping script for configuring and starting up Spark master/worker nodes
* A wrapper around the `spark-submit` CLI to submit a Spark application


After the container build and push process is complete, use the Amazon SageMaker Python SDK to submit a managed, distributed Spark application that performs our dataset processing.

Build the example Spark container.

In [4]:
!pygmentize container/Dockerfile

FROM openjdk:8-jre-slim

RUN apt-get update
RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
RUN apt-get clean
RUN rm -rf /var/lib/apt/lists/*

# http://blog.stuart.axelbrooke.com/python-3-on-spark-return-of-the-pythonhashseed
ENV PYTHONHASHSEED 0
ENV PYTHONIOENCODING UTF-8
ENV PIP_DISABLE_PIP_VERSION_CHECK 1

# Install Hadoop
ENV HADOOP_VERSION 3.2.1
ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
ENV HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
ENV PATH $PATH:$HADOOP_HOME/bin
RUN curl -sL --retry 10 \
  "http://archive.apache.org/dist/hadoop/common/hadoop-$HADOOP_VERSION/hadoop-$HADOOP_VERSION.tar.gz" \
  | gunzip \
  | tar -x -C /usr/ \
 && rm -rf $HADOOP_HOME/share/doc \
 && chown -R root:root $HADOOP_HOME

# Install Spark
ENV SPARK_VERSION 2.4.6
ENV SPARK_PACKAGE spark-${SPARK_VERSION}-bin-without-hadoop
ENV SPARK_HOME /usr/spark-${SPARK_VERSION}
ENV SPARK_DIST_CLASSPATH="$HADOOP_

In [5]:
docker_repo = 'amazon-reviews-spark-processor'
docker_tag = 'latest'

In [6]:
!docker build -t $docker_repo:$docker_tag -f container/Dockerfile ./container

Sending build context to Docker daemon  4.441MB
Step 1/37 : FROM openjdk:8-jre-slim
 ---> d2f9f3c77c25
Step 2/37 : RUN apt-get update
 ---> Using cache
 ---> cf302be67c70
Step 3/37 : RUN apt-get install -y curl unzip python3 python3-setuptools python3-pip python-dev python3-dev python-psutil
 ---> Using cache
 ---> 62d56031da94
Step 4/37 : RUN pip3 install py4j psutil==5.6.5 numpy==1.17.4
 ---> Using cache
 ---> 381148b87d0e
Step 5/37 : RUN apt-get clean
 ---> Using cache
 ---> f0ec1306ff8a
Step 6/37 : RUN rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> dba6330dec66
Step 7/37 : ENV PYTHONHASHSEED 0
 ---> Using cache
 ---> 4a8cdcf14e77
Step 8/37 : ENV PYTHONIOENCODING UTF-8
 ---> Using cache
 ---> 911008bea772
Step 9/37 : ENV PIP_DISABLE_PIP_VERSION_CHECK 1
 ---> Using cache
 ---> ede4fd31963b
Step 10/37 : ENV HADOOP_VERSION 3.2.1
 ---> Using cache
 ---> f844be4ebfd9
Step 11/37 : ENV HADOOP_HOME /usr/hadoop-$HADOOP_VERSION
 ---> Using cache
 ---> d36f503d5444
Step 12/37 : ENV HADOOP

# Check the Docker Image
If the image did not build properly, re-run the cell above.

In [7]:
!docker inspect $docker_repo:$docker_tag

[
    {
        "Id": "sha256:8671534dc1ce8fffdfd87980dc56e9b65a113b55ee8c8b154980c826c6667258",
        "RepoTags": [
            "amazon-reviews-spark-processor:latest"
        ],
        "RepoDigests": [],
        "Parent": "sha256:e09915b6efb9e4238d028b7fce0ff09e800b5fc9a9f5e7424155a796c788ab3a",
        "Comment": "",
        "Created": "2020-08-22T18:44:30.675788897Z",
        "Container": "db21ac77b9afa3852e86075baa270e72acc7d64e3b67cbcf996e4b9cf1f1bf73",
        "ContainerConfig": {
            "Hostname": "db21ac77b9af",
            "Domainname": "",
            "User": "",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "Tty": false,
            "OpenStdin": false,
            "StdinOnce": false,
            "Env": [
                "PATH=/usr/bin:/opt/program:/usr/local/openjdk-8/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/hadoop-3.2.1/bin:/usr/spark-2.4.6/bin",
                "LAN

# Push the Image to a Private Docker Repo (Amazon ECR)

In [8]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

250107111215.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor:latest


In [9]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


### Ignore the `RepositoryNotFoundException` Error Below

In [10]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo


An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'amazon-reviews-spark-processor' does not exist in the registry with id '250107111215'
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:250107111215:repository/amazon-reviews-spark-processor",
        "registryId": "250107111215",
        "repositoryName": "amazon-reviews-spark-processor",
        "repositoryUri": "250107111215.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor",
        "createdAt": 1598121877.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [11]:
!docker tag $docker_repo:$docker_tag $image_uri

In [12]:
!docker push $image_uri

The push refers to repository [250107111215.dkr.ecr.us-west-2.amazonaws.com/amazon-reviews-spark-processor]

27d0bf81: Preparing 
b8554a60: Preparing 
4f1898a0: Preparing 
37afa9f5: Preparing 
8fe2ab9b: Preparing 
b293986c: Preparing 
1bfb9160: Preparing 
a8d94da8: Preparing 
5a4e7378: Preparing 
c0d85376: Preparing 
aa6518ef: Preparing 
6348f103: Preparing 
99d2e675: Preparing 
35ba2eea: Preparing 
ffc3afea: Preparing 
8102614d: Preparing 
52ed4cbd: Preparing 
0fa5728d: Preparing 


7d0bf81: Pushing  1.909GB/2.012GBPushing  209.8MB/481.8MBPushing   53.2MB/69.22MB

7d0bf81: Pushed   2.022GB/2.012GBlatest: digest: sha256:e1f613fc0d007d743bc451ce0d76bb3ca5e2d5ce66550e4fa0c784afbde9403a size: 4318


# Run the Job using a SageMaker Processing Job

Next, use the Amazon SageMaker Python SDK to submit a processing job. Use the Spark container that was just built, and a Spark ML script for processing in the job configuration.

Review the Spark processing script.

In [13]:
!pygmentize preprocess-spark-text-to-bert.py

from __future__ import print_function
from __future__ import unicode_literals

import time
import sys
import os
import shutil
import csv
import collections
import subprocess
import sys
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'pip', '--upgrade'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'wrapt', '--upgrade', '--ignore-installed'])
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow==2.1.0', '--ignore-installed'])
import tensorflow as tf
print(tf.__version__)
#subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'transformers==2.8.0'])
from transformers import DistilBertTokenizer

import pyspark
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import split
from pyspark.sql.functions import udf, col
from pyspark.sql.types import *

tokenizer = DistilBertTokenizer.from_pretrained('distil

In [14]:
from sagemaker.processing import ScriptProcessor

processor = ScriptProcessor(base_job_name='spark-amazon-reviews-processor',
                            image_uri=image_uri,
                            command=['/opt/program/submit'],
                            role=role,
                            instance_count=2, # instance_count needs to be > 1 or you will see the following error:  "INFO yarn.Client: Application report for application_ (state: ACCEPTED)"
                            instance_type='ml.r5.xlarge',
                            env={'mode': 'python'},
                            max_runtime_in_seconds=7200)

# Setup Output Data

In [15]:
from time import gmtime, strftime
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

output_prefix = 'amazon-reviews-spark-processor-{}'.format(timestamp_prefix)

In [16]:
train_data_bert_output = 's3://{}/{}/output/bert-train'.format(bucket, output_prefix)
validation_data_bert_output = 's3://{}/{}/output/bert-validation'.format(bucket, output_prefix)
test_data_bert_output = 's3://{}/{}/output/bert-test'.format(bucket, output_prefix)

print(train_data_bert_output)
print(validation_data_bert_output)
print(test_data_bert_output)

s3://sagemaker-us-west-2-250107111215/amazon-reviews-spark-processor-2020-08-22-18-46-02/output/bert-train
s3://sagemaker-us-west-2-250107111215/amazon-reviews-spark-processor-2020-08-22-18-46-02/output/bert-validation
s3://sagemaker-us-west-2-250107111215/amazon-reviews-spark-processor-2020-08-22-18-46-02/output/bert-test


In [17]:
from sagemaker.processing import ProcessingOutput

processor.run(code='preprocess-spark-text-to-bert.py',
              arguments=['s3_input_data', s3_input_data,
                         's3_output_train_data', train_data_bert_output,
                         's3_output_validation_data', validation_data_bert_output,
                         's3_output_test_data', test_data_bert_output,                         
              ],
              # We need to define these outputs so we can later call  
              #    ProcessingJob.from_processing_name() 
              # to describe the job and poll for Completed status
              outputs=[
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-train',
                                        source='/opt/ml/processing/output/bert/train'),
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-validation',
                                        source='/opt/ml/processing/output/bert/validation'),
                       ProcessingOutput(s3_upload_mode='EndOfJob',
                                        output_name='bert-test',
                                        source='/opt/ml/processing/output/bert/test'),
              ],          
              logs=True,
              wait=False
)

Parameter 'session' will be renamed to 'sagemaker_session' in SageMaker Python SDK v2.



Job Name:  spark-amazon-reviews-processor-2020-08-22-18-46-02-060
Inputs:  [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-250107111215/spark-amazon-reviews-processor-2020-08-22-18-46-02-060/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'bert-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-250107111215/spark-amazon-reviews-processor-2020-08-22-18-46-02-060/output/bert-train', 'LocalPath': '/opt/ml/processing/output/bert/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-validation', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-250107111215/spark-amazon-reviews-processor-2020-08-22-18-46-02-060/output/bert-validation', 'LocalPath': '/opt/ml/processing/output/bert/validation', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-test', 'S3Output': {'S3Uri

In [18]:
spark_processing_job_name = processor.jobs[-1].describe()['ProcessingJobName']



In [19]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(region, spark_processing_job_name)))


In [20]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(region, spark_processing_job_name)))


In [21]:
from IPython.core.display import display, HTML

# This is different than the job name because we are not using ProcessingOutput's in this Spark ML case.
spark_processing_job_s3_output_prefix = output_prefix

display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(bucket, spark_processing_job_s3_output_prefix, region)))


# Please Wait Until the Processing Job Completes
Re-run this next cell until the job status shows `Completed`.

In [22]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(processing_job_name=spark_processing_job_name,
                                                                            sagemaker_session=sagemaker_session)

processing_job_description = running_processor.describe()

processing_job_status = processing_job_description['ProcessingJobStatus']
print('\n')
print(processing_job_status)
print('\n')

print(processing_job_description)



InProgress


{'ProcessingInputs': [{'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-250107111215/spark-amazon-reviews-processor-2020-08-22-18-46-02-060/input/code/preprocess-spark-text-to-bert.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}], 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 'bert-train', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-250107111215/spark-amazon-reviews-processor-2020-08-22-18-46-02-060/output/bert-train', 'LocalPath': '/opt/ml/processing/output/bert/train', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-validation', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-250107111215/spark-amazon-reviews-processor-2020-08-22-18-46-02-060/output/bert-validation', 'LocalPath': '/opt/ml/processing/output/bert/validation', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'bert-test', 'S3Output': {'S3Uri': 's3://sa

# _Please Wait Until the ^^ Processing Job ^^ Completes Above._

In [23]:
running_processor.wait()

2020-08-22 18:50:10,957 INFO namenode.NameNode: STARTUP_MSG: 
/************************************************************
STARTUP_MSG: Starting NameNode
STARTUP_MSG:   host = algo-1/10.0.135.84
STARTUP_MSG:   args = [-format, -force]
STARTUP_MSG:   version = 3.2.1
STARTUP_MSG:   classpath = /usr/hadoop-3.2.1/etc/hadoop:/usr/hadoop-3.2.1/share/hadoop/common/lib/asm-5.0.4.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/commons-logging-1.1.3.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jetty-webapp-9.3.24.v20180605.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jetty-io-9.3.24.v20180605.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/j2objc-annotations-1.1.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/curator-framework-2.13.0.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/jackson-jaxrs-1.9.13.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/accessors-smart-1.2.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/curator-client-2.13.0.jar:/usr/hadoop-3.2.1/share/hadoop/common/lib/netty-3.10.5.Final.jar

Starting nodemanagers
localhost: /usr/hadoop-3.2.1/bin/../libexec/hadoop-functions.sh: line 982: ssh: command not found
2020-08-22 18:50:24,332 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2020-08-22 18:50:25.270025: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-08-22 18:50:25.270119: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-08-22 18:50:25.270132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2.1.0
#

2020-08-22 18:50:32,193 INFO yarn.Client: Submitting application application_1598122222765_0001 to ResourceManager
2020-08-22 18:50:32,385 INFO impl.YarnClientImpl: Submitted application application_1598122222765_0001
2020-08-22 18:50:32,387 INFO cluster.SchedulerExtensionServices: Starting Yarn extension services with app application_1598122222765_0001 and attemptId None
2020-08-22 18:50:33,395 INFO yarn.Client: Application report for application_1598122222765_0001 (state: ACCEPTED)
2020-08-22 18:50:33,399 INFO yarn.Client: 
#011 client token: N/A
#011 diagnostics: AM container is launched, waiting for AM container to Register with RM
#011 ApplicationMaster host: N/A
#011 ApplicationMaster RPC port: -1
#011 queue: default
#011 start time: 1598122232289
#011 final status: UNDEFINED
#011 tracking URL: http://algo-1:8088/proxy/application_1598122222765_0001/
#011 user: root
2020-08-22 18:50:34,401 INFO yarn.Client: Application report for application_1598122222765_0001 (state: ACCEPTED)
2

2020-08-22 18:51:02,184 INFO storage.BlockManagerInfo: Added broadcast_0_piece0 in memory on algo-2:35053 (size: 42.8 KB, free: 11.9 GB)
2020-08-22 18:51:04,198 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 0.0 (TID 0) in 2860 ms on algo-2 (executor 1) (1/1)
2020-08-22 18:51:04,200 INFO cluster.YarnScheduler: Removed TaskSet 0.0, whose tasks have all completed, from pool 
2020-08-22 18:51:04,206 INFO scheduler.DAGScheduler: ResultStage 0 (showString at NativeMethodAccessorImpl.java:0) finished in 2.971 s
2020-08-22 18:51:04,210 INFO scheduler.DAGScheduler: Job 0 finished: showString at NativeMethodAccessorImpl.java:0, took 3.002318 s
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|help

2020-08-22 18:51:12,173 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 5.0 (TID 5) in 2779 ms on algo-2 (executor 1) (1/1)
2020-08-22 18:51:12,173 INFO cluster.YarnScheduler: Removed TaskSet 5.0, whose tasks have all completed, from pool 
2020-08-22 18:51:12,174 INFO python.PythonAccumulatorV2: Connected to AccumulatorServer at host: 127.0.0.1 port: 35081
2020-08-22 18:51:12,175 INFO scheduler.DAGScheduler: ResultStage 5 (showString at NativeMethodAccessorImpl.java:0) finished in 2.795 s
2020-08-22 18:51:12,176 INFO scheduler.DAGScheduler: Job 5 finished: showString at NativeMethodAccessorImpl.java:0, took 2.797418 s
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

2020-08-22 19:00:37,479 INFO scheduler.TaskSetManager: Finished task 1.0 in stage 7.0 (TID 8) in 562782 ms on algo-2 (executor 1) (1/2)
2020-08-22 19:04:10,284 INFO scheduler.TaskSetManager: Finished task 0.0 in stage 7.0 (TID 7) in 775587 ms on algo-2 (executor 1) (2/2)
2020-08-22 19:04:10,284 INFO cluster.YarnScheduler: Removed TaskSet 7.0, whose tasks have all completed, from pool 
2020-08-22 19:04:10,285 INFO scheduler.DAGScheduler: ResultStage 7 (runJob at SparkHadoopWriter.scala:78) finished in 775.698 s
2020-08-22 19:04:10,285 INFO scheduler.DAGScheduler: Job 7 finished: runJob at SparkHadoopWriter.scala:78, took 775.704249 s
2020-08-22 19:04:10,305 INFO io.SparkHadoopWriter: Job job_20200822185114_0038 committed.
Wrote to output file:  /opt/ml/processing/output/bert/train
2020-08-22 19:04:10,347 INFO datasources.FileSourceStrategy: Pruning directories with: 
2020-08-22 19:04:10,347 INFO datasources.FileSourceStrategy: Post-Scan Filters: AtLeastNNulls(n, review_body#13)
2020-08-

2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 226
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 235
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 245
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 272
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 274
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 257
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 247
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 254
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 240
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 263
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 249
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 259
2020-08-22 19:19:33,125 INFO spark.ContextCleaner: Cleaned accumulator 233
2020-08-22 19:19:33,125 I

2020-08-22 19:29:56
Finished Yarn configuration files setup.

Received end of job signal, exiting...
Finished Yarn configuration files setup.




# Inspect the Processed Output Dataset


## _The next cells will not work properly until the job completes above._


Take a look at a few rows of the transformed dataset to make sure the processing was successful.

In [24]:
!aws s3 ls --recursive $train_data_bert_output/

In [25]:
!aws s3 ls --recursive $validation_data_bert_output/

In [26]:
!aws s3 ls --recursive $test_data_bert_output/

In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();